In [19]:
import numpy as np
import pandas as pd

In [20]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [21]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA3/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [22]:
mineralogy = mineralogy.dropna()

In [23]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [24]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [25]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [26]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
11189,69.561517,0.388008,15.622604,2.122984,2.143501,0.064385,1.011814,2.283311,3.436310,3.365567
11190,69.522275,0.389983,15.631294,2.129026,2.151350,0.064560,1.019471,2.296393,3.435287,3.360361
11191,69.483662,0.391940,15.639882,2.134586,2.159186,0.064733,1.027078,2.309388,3.434276,3.355268
11192,69.447395,0.393790,15.647970,2.139502,2.166635,0.064895,1.034285,2.321695,3.433325,3.350509
11596,69.650685,0.383281,15.600488,2.117609,2.123095,0.063982,0.993542,2.251500,3.438603,3.377215


In [29]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
11189,69.561517,0.388008,15.622604,2.122984,2.143501,0.064385,1.011814,2.283311,3.436310,3.365567
11190,69.522275,0.389983,15.631294,2.129026,2.151350,0.064560,1.019471,2.296393,3.435287,3.360361
11191,69.483662,0.391940,15.639882,2.134586,2.159186,0.064733,1.027078,2.309388,3.434276,3.355268
11192,69.447395,0.393790,15.647970,2.139502,2.166635,0.064895,1.034285,2.321695,3.433325,3.350509
11596,69.650685,0.383281,15.600488,2.117609,2.123095,0.063982,0.993542,2.251500,3.438603,3.377215
...,...,...,...,...,...,...,...,...,...,...
160540,66.497094,0.581065,16.359982,1.041828,3.348761,0.074383,1.867639,3.795285,3.337224,3.096739
160541,66.540386,0.578607,16.354418,1.048972,3.333654,0.074272,1.855534,3.775169,3.338801,3.100188
160542,66.614684,0.574385,16.344735,1.060516,3.308182,0.074075,1.834838,3.740863,3.341484,3.106239
160543,66.605811,0.574921,16.346301,1.052717,3.314717,0.074066,1.837613,3.746503,3.341056,3.106294


In [30]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:135000]

In [31]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA3/area3_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA3/area3_2.txt", dataset_name="Vistelius (1995)")
#cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA3/area3_3.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [14]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3/area3_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3/area3_2_results_excel.txt")
#df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3/area3_3_results_excel.txt")

In [16]:
df_final = pd.concat([df_final1, df_final2])

In [17]:
df_final

,Q,P,K
SAMPLE,,,
24087,29.516720,37.510518,22.790947
24088,29.484957,37.543130,22.772602
24089,29.453165,37.575692,22.754435
24698,29.557388,37.469844,22.809065
24699,29.521168,37.506955,22.788288
...,...,...,...
360180,21.300478,45.212225,20.763760
360784,22.386469,44.136415,21.160115
360785,22.360367,44.162673,21.149665


## QAPF classification

In [18]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [19]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite'], dtype=object)

In [20]:
df_final.to_excel("../_CIPW/CIPW/AREA3/qapf.xlsx", index=True)

-------

In [22]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA3/QAPF_counts.xlsx")

-----

In [3]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area_subdivided.xlsx", index_col=[0], usecols = [0, 1, 2, 3, 4, 29])

In [6]:
Area

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area
2,-5.134927e+05,6.119173e+06,53,T,1
6,-9.966614e+05,5.836313e+06,53,T,1
9,-1.061190e+06,5.804252e+06,53,T,1
10,-9.953983e+05,5.877750e+06,53,T,1
11,-1.202844e+06,5.901332e+06,53,T,1
...,...,...,...,...,...
4562,2.276617e+08,-4.667076e+08,53,T,5
4594,1.898774e+08,-4.145875e+08,53,T,5
4628,1.947005e+08,-4.390290e+08,53,T,5
4643,1.930846e+08,-4.358868e+08,53,T,5


In [24]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,7.880625e+05,4.936585e+06,53.0,V,2.0,52.526539,17.979983,22.933498,monzo granite
2,-5.134927e+05,6.119173e+06,53.0,V,1.0,45.821201,19.425633,32.374005,monzo granite
3,1.544245e+06,7.110278e+06,53.0,V,3.0,50.716606,6.475010,37.375668,syeno granite
4,1.083775e+06,6.877413e+06,53.0,V,3.0,41.397182,23.234657,27.592905,monzo granite
5,1.641085e+06,7.925964e+06,53.0,V,4.0,54.609970,18.013528,19.746868,monzo granite
...,...,...,...,...,...,...,...,...,...
4654,6.202597e+05,5.148324e+06,53.0,V,2.0,0.000000,52.913423,17.411244,monzodiorite monzogabbro
4655,1.756745e+06,7.702655e+06,53.0,V,4.0,0.000000,59.865509,15.514612,monzodiorite monzogabbro
4656,6.586139e+05,5.200781e+06,53.0,V,2.0,0.000000,52.355599,12.039131,monzodiorite monzogabbro
4658,1.177291e+06,6.811066e+06,53.0,V,3.0,0.000000,71.436466,10.147088,monzodiorite monzogabbro


In [26]:
QAPF_control_area3 = QAPF_control[QAPF_control['area'] ==3]
QAPF_control_area3.to_excel("../_CIPW/CIPW/AREA3/QAPF_control_points.xlsx", index=True)

In [27]:
QAPF_control_area3

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
3,1.544245e+06,7.110278e+06,53.0,V,3.0,50.716606,6.475010,37.375668,syeno granite
4,1.083775e+06,6.877413e+06,53.0,V,3.0,41.397182,23.234657,27.592905,monzo granite
12,1.257863e+06,6.881257e+06,53.0,V,3.0,46.418781,23.574272,26.292937,monzo granite
15,1.062527e+06,7.781132e+06,53.0,V,3.0,41.948841,25.957568,29.502100,monzo granite
18,7.552345e+05,7.733147e+06,53.0,V,3.0,38.498253,29.828145,28.683326,monzo granite
...,...,...,...,...,...,...,...,...,...
4645,1.094202e+06,7.789042e+06,53.0,V,3.0,0.000000,45.096558,24.944502,monzonite
4648,1.299501e+06,6.808221e+06,53.0,V,3.0,9.464496,56.033000,11.319685,quartz monzodiorite\nquartz monzogabbro
4652,1.766361e+06,7.168888e+06,53.0,V,3.0,2.939407,57.054470,12.755601,monzodiorite monzogabbro
4653,1.321263e+06,6.795969e+06,53.0,V,3.0,6.284857,60.868792,8.390470,quartz monzodiorite\nquartz monzogabbro
